In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import spacy
import pickle
import torch.nn.functional as F

In [24]:
df = pd.read_excel("Product Matching Phase 1\Product Matching Phase 1\Product Matching Dataset.xlsx", 'Dataset')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Ahmed Rabie\AppData\Local\Temp\ipykernel_4256\2690156247.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_excel("Product Matching Phase 1\Product Matching Phase 1\Product Matching Dataset.xlsx", 'Dataset')


In [25]:
df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [26]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["marketplace_product_name_ar"])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["seller_item_name"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

num_classes = len(label_encoder.classes_)   

In [27]:
nlp = spacy.blank("ar")

In [28]:
def arabic_tokenizer(text):
    return [token.text for token in nlp(text)]

def yield_tokens(texts):
    for text in texts:
        yield arabic_tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_texts), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

def text_to_tensor(text, max_len=50):
    tokens = arabic_tokenizer(text)[:max_len]
    token_ids = [vocab[token] for token in tokens]
    return torch.tensor(token_ids + [vocab["<pad>"]] * (max_len - len(token_ids)), dtype=torch.long)

train_inputs = torch.stack([text_to_tensor(text) for text in train_texts])
val_inputs = torch.stack([text_to_tensor(text) for text in val_texts])
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)

In [29]:
class ArabicTextDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {"input_ids": self.inputs[idx], "label": self.labels[idx]}

In [30]:
batch_size = 32
train_dataset = ArabicTextDataset(train_inputs, train_labels)
val_dataset = ArabicTextDataset(val_inputs, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [31]:
class SimpleTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, num_classes):
        super(SimpleTransformerClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, 50, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1), :]
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  
        x = self.dropout(x)
        return self.fc(x)

embed_dim = 128   
num_heads = 4     
hidden_dim = 256  
num_layers = 2    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleTransformerClassifier(
    vocab_size=len(vocab),
    embed_dim=embed_dim,
    num_heads=num_heads,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes
).to(device)

C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4)

def train(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss, correct = 0, 0

    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(dataloader), correct / len(dataloader.dataset)


def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss, correct = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids)
            loss = loss_fn(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(dataloader), correct / len(dataloader.dataset)


In [33]:
device

device(type='cuda')

In [37]:
epochs = 3

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_acc = evaluate(model, val_loader, loss_fn, device)

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


Epoch 1:
  Train Loss: 0.2251, Train Acc: 0.9638
  Val Loss: 0.2471, Val Acc: 0.9590
Epoch 2:
  Train Loss: 0.1980, Train Acc: 0.9675
  Val Loss: 0.2387, Val Acc: 0.9596
Epoch 3:
  Train Loss: 0.1718, Train Acc: 0.9718
  Val Loss: 0.2341, Val Acc: 0.9612


In [38]:
torch.save(model.state_dict(), "simple_transformer_arabic.pth")

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [39]:
def classify_text(text):

    model = SimpleTransformerClassifier(
    vocab_size=len(vocab),
    embed_dim=embed_dim,
    num_heads=num_heads,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes
    ).to(device)
    
    model.load_state_dict(torch.load("simple_transformer_arabic.pth"))

    input_tensor = text_to_tensor(text).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)   
        confidence, predicted_class = torch.max(probabilities, dim=1)   
    with open("label_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)

    predicted_label = label_encoder.inverse_transform([predicted_class.item()])[0]
    confidence_score = confidence.item() * 100   

    return predicted_label, confidence_score


text_sample = 'باوريكتا اقراص'
predicted_label = classify_text(text_sample)
print(f"Predicted Class: {predicted_label}")

Predicted Class: ('باوريكتا 20 مجم 4 قرص', 97.24669456481934)


C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
